<a href="https://colab.research.google.com/github/weedge/doraemon-nb/blob/main/achat_miniCPMo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# install

In [ ]:
!cd /content && rm -rf achatbot && git clone https://github.com/ai-bot-pro/achatbot.git -b feat/vision_voice

In [41]:
%cd /content/achatbot

/content/achatbot


In [ ]:
!git submodule update --init --recursive

In [ ]:
!bash scripts/pypi_achatbot.sh dev

In [ ]:
!pip install -q dist/achatbot-0.0.8.10.11-py3-none-any.whl[llm_transformers_manual_vision_voice_minicpmo,grpc]

# Prepare code and install AutoGPTQ


if use openbmb/MiniCPM-o-2_6-int4, need use AutoGPTQ lib

In [36]:
%cd /content

/content


In [ ]:
!git clone https://github.com/OpenBMB/AutoGPTQ.git -b minicpmo
!cd AutoGPTQ && git checkout minicpmo && pip install -vvv --quiet --no-build-isolation -e .

# download ckpt


In [6]:
from google.colab import userdata
HF_TOKEN=userdata.get('HF_TOKEN')

In [ ]:
!huggingface-cli login --token $HF_TOKEN --add-to-git-credential


In [ ]:
!huggingface-cli download FunAudioLLM/SenseVoiceSmall --quie --local-dir /content/models/FunAudioLLM/SenseVoiceSmall


In [8]:
!huggingface-cli download openbmb/MiniCPM-o-2_6 --quiet --local-dir /content/models/openbmb/MiniCPM-o-2_6


/content/models/openbmb/MiniCPM-o-2_6


In [38]:
!huggingface-cli download openbmb/MiniCPM-o-2_6-int4 --quiet --local-dir /content/models/openbmb/MiniCPM-o-2_6-int4

/content/models/openbmb/MiniCPM-o-2_6-int4


# test tts

In [ ]:
!LLM_MODEL_NAME_OR_PATH=/content/models/openbmb/MiniCPM-o-2_6 \
    LLM_GEN_TEMPERATURE=0.0 \
    LLM_DEVICE=cuda LLM_TORCH_DTYPE=bfloat16 \
    TTS_TEXT="你叫什么名字?" \
    python -m unittest test.modules.speech.tts.test_minicpmo.TestMiniCPMoTTS.test_synthesize

In [ ]:
from IPython.display import Audio
Audio("/content/achatbot/records/test_tts_minicpmo.wav",autoplay=True)

# tts speaker with grpc


In [ ]:
#!python -m src.cmd.grpc.speaker.server.serve

In [ ]:
!TTS_TAG=tts_minicpmo \
    LLM_DEVICE=cuda LLM_GEN_TEMPERATURE=0.5 LLM_TORCH_DTYPE=bfloat16 \
    LLM_MODEL_NAME_OR_PATH=/content/models/openbmb/MiniCPM-o-2_6 \
    IS_SAVE=1 IS_RELOAD=1 python -m src.cmd.grpc.speaker.client

2025-02-14 03:47:37,149 - chat-bot-tts-client - DEBUG - /content/achatbot/src/cmd/grpc/speaker/client.py:59 - load_model - tts_tag: "tts_minicpmo"
is_reload: true
json_kwargs: "{\"lm_model_name_or_path\": \"/content/models/openbmb/MiniCPM-o-2_6\", \"lm_device_map\": null, \"lm_device\": \"cuda\", \"lm_torch_dtype\": \"bfloat16\", \"lm_attn_impl\": null, \"user_role\": \"user\", \"warnup_prompt\": \"Repeat the word \'weedge niu bi\'.\", \"warnup_steps\": 1, \"init_chat_role\": \"system\", \"init_chat_prompt\": \"\", \"lm_gen_seed\": 42, \"lm_gen_max_new_tokens\": 1024, \"lm_gen_min_new_tokens\": 0, \"lm_gen_do_sample\": false, \"lm_gen_temperature\": 0.5, \"lm_gen_top_k\": 1, \"lm_gen_top_p\": 0.8, \"lm_gen_repetition_penalty\": 1.0, \"lm_tokenizer_decode_batch_size\": 60, \"chat_history_size\": 10, \"lm_stream\": true, \"model_type\": \"chat_completion\", \"lm_bnb_quant_type\": \"int4\", \"instruct_tpl\": \"\"}"

2025-02-14 03:47:37,149 - chat-bot-tts-client - DEBUG - /content/achatbot

In [ ]:
from IPython.display import Audio
Audio("/content/achatbot/records/grpc_tts_minicpmo.wav",autoplay=True)


# webrtc chat bot

In [ ]:
!pip install -q "dist/achatbot-0.0.8.10.11-py3-none-any.whl[fastapi_bot_server,daily_room_audio_stream,livekit_room_audio_stream,silero_vad_analyzer,sense_voice_asr,llm_transformers_manual_vision_voice_minicpmo,queue]"


In [ ]:
# if use flash attention 2
!pip install -q --no-build-isolation "dist/achatbot-0.0.8.10.11-py3-none-any.whl[flash-attn]"

In [11]:
from google.colab import userdata
DAILY_API_KEY=userdata.get('DAILY_API_KEY')

LIVEKIT_URL=userdata.get('LIVEKIT_URL')
LIVEKIT_API_KEY=userdata.get('LIVEKIT_API_KEY')
LIVEKIT_API_SECRET=userdata.get('LIVEKIT_API_SECRET')

In [42]:
%cd /content/achatbot


/content/achatbot


## voice webrtc chat bot

In [30]:
!cat /content/daily_minicpmo_voice_bot.json

{
  "chat_bot_name": "DailyMiniCPMoVoiceBot",
  "room_name": "chat-room",
  "room_url": "",
  "token": "",
  "room_manager": {
    "tag": "daily_room",
    "args": {
      "privacy": "public"
    }
  },
  "services": {
    "pipeline": "achatbot",
    "vad": "silero",
    "voice_llm": "minicpmo"
  },
  "config": {
    "vad": {
      "tag": "silero_vad_analyzer",
      "args": { "stop_secs": 0.7 }
    },
    "voice_llm": {
      "tag": "glm_voice_processor",
      "args": {
        "lm_device": "cuda",
        "lm_torch_dtype": "bfloat16",
        "lm_attn_impl": "flash_attention_2",
        "warnup_steps": 1,
        "lm_gen_temperature": 0.5,
        "ref_audio_path":"/content/achatbot/test/audio_files/asr_example_zh.wav",
        "lm_model_name_or_path": "/content/models/openbmb/MiniCPM-o-2_6"
      }
    }
  },
  "config_list": []
}


In [ ]:
!DAILY_API_KEY=$DAILY_API_KEY \
  python -m src.cmd.bots.main -f /content/daily_minicpmo_voice_bot.json


In [57]:
!DAILY_API_KEY=$DAILY_API_KEY \
  python -m src.cmd.bots.main -f /content/daily_asr_minicpmo_voice_bot.json


{
    "chat_bot_name": "DailyAsrMiniCPMoVoiceBot",
    "config": {
        "asr": {
            "args": {
                "language": "zn",
                "model_name_or_path": "/content/models/FunAudioLLM/SenseVoiceSmall"
            },
            "tag": "sense_voice_asr"
        },
        "vad": {
            "args": {
                "stop_secs": 0.7
            },
            "tag": "silero_vad_analyzer"
        },
        "voice_llm": {
            "args": {
                "lm_attn_impl": "flash_attention_2",
                "lm_device": "cuda",
                "lm_gen_temperature": 0.5,
                "lm_model_name_or_path": "/content/models/openbmb/MiniCPM-o-2_6",
                "lm_torch_dtype": "bfloat16",
                "ref_audio_path": "/content/achatbot/test/audio_files/asr_example_zh.wav",
                "warnup_steps": 1
            },
            "tag": "minicpmo_voice_processor"
        }
    },
    "config_list": [],
    "room_manager": {
        "args": {
  

In [42]:
from pydub import AudioSegment
import glob

# 获取所有需要合并的音频文件路径
audio_files = glob.glob('/content/achatbot/records/bot_speak_2025-02-14_09-50-50.*.wav')
print(audio_files)

# 初始化一个空的 AudioSegment 对象
combined_sound = AudioSegment.empty()

# 循环遍历所有音频文件并合并
for file in audio_files:
    sound = AudioSegment.from_file(file, format="wav")
    combined_sound += sound

# 定义输出文件路径
output_file = '/content/achatbot/records/merged_audio.wav'

# 导出合并后的音频文件
combined_sound.export(output_file, format="wav")

print(f"音频文件已合并到 {output_file}")

['/content/achatbot/records/bot_speak_2025-02-14_09-50-50.128.wav', '/content/achatbot/records/bot_speak_2025-02-14_09-50-50.125.wav', '/content/achatbot/records/bot_speak_2025-02-14_09-50-50.126.wav', '/content/achatbot/records/bot_speak_2025-02-14_09-50-50.131.wav', '/content/achatbot/records/bot_speak_2025-02-14_09-50-50.130.wav', '/content/achatbot/records/bot_speak_2025-02-14_09-50-50.127.wav', '/content/achatbot/records/bot_speak_2025-02-14_09-50-50.124.wav', '/content/achatbot/records/bot_speak_2025-02-14_09-50-50.123.wav', '/content/achatbot/records/bot_speak_2025-02-14_09-50-50.129.wav']
音频文件已合并到 /content/achatbot/records/merged_audio.wav


In [43]:
from IPython.display import Audio
Audio("/content/achatbot/records/merged_audio.wav",autoplay=True)

## omni webrtc chat bot

In [33]:
!cat /content/daily_minicpmo_vision_voice_bot.json

{
  "chat_bot_name": "DailyMiniCPMoVisionVoiceBot",
  "room_name": "chat-room",
  "room_url": "",
  "token": "",
  "room_manager": {
    "tag": "daily_room",
    "args": {
      "privacy": "public"
    }
  },
  "services": {
    "pipeline": "achatbot",
    "vad": "silero",
    "omni_llm": "minicpmo"
  },
  "config": {
    "vad": {
      "tag": "silero_vad_analyzer",
      "args": { "stop_secs": 0.7 }
    },
    "omni_llm": {
      "tag": "minicpmo_vision_voice_processor",
      "args": {
        "lm_device_map": "auto",
        "lm_torch_dtype": "bfloat16",
        "lm_attn_impl": "flash_attention_2",
        "warnup_steps": 1,
        "lm_gen_temperature": 0.5,
        "lm_model_name_or_path": "/content/models/openbmb/MiniCPM-o-2_6"
      }
    }
  },
  "config_list": []
}


In [35]:
!!DAILY_API_KEY=$DAILY_API_KEY \
  python -m src.cmd.bots.main -f /content/daily_minicpmo_vision_voice_bot.json


['{',
 '    "chat_bot_name": "DailyMiniCPMoVisionVoiceBot",',
 '    "config": {',
 '        "omni_llm": {',
 '            "args": {',
 '                "lm_attn_impl": "flash_attention_2",',
 '                "lm_device_map": "auto",',
 '                "lm_gen_temperature": 0.5,',
 '                "lm_model_name_or_path": "/content/models/openbmb/MiniCPM-o-2_6",',
 '                "lm_torch_dtype": "bfloat16",',
 '                "warnup_steps": 1',
 '            },',
 '            "tag": "minicpmo_vision_voice_processor"',
 '        },',
 '        "vad": {',
 '            "args": {',
 '                "stop_secs": 0.7',
 '            },',
 '            "tag": "silero_vad_analyzer"',
 '        }',
 '    },',
 '    "config_list": [],',
 '    "room_manager": {',
 '        "args": {',
 '            "privacy": "public"',
 '        },',
 '        "tag": "daily_room"',
 '    },',
 '    "room_name": "chat-room",',
 '    "room_url": "",',
 '    "services": {',
 '        "omni_llm": "minicpm

In [39]:
# use gptq int4 ckpt
!cat /content/daily_minicpmo_vision_voice_bot.json

{
  "chat_bot_name": "DailyMiniCPMoVisionVoiceBot",
  "room_name": "chat-room",
  "room_url": "",
  "token": "",
  "room_manager": {
    "tag": "daily_room",
    "args": {
      "privacy": "public"
    }
  },
  "services": {
    "pipeline": "achatbot",
    "vad": "silero",
    "omni_llm": "minicpmo"
  },
  "config": {
    "vad": {
      "tag": "silero_vad_analyzer",
      "args": { "stop_secs": 0.7 }
    },
    "omni_llm": {
      "tag": "minicpmo_vision_voice_processor",
      "args": {
        "lm_device_map": "auto",
        "lm_torch_dtype": "bfloat16",
        "lm_attn_impl": "flash_attention_2",
        "warnup_steps": 1,
        "lm_gen_temperature": 0.5,
        "use_gptq_ckpt": true,
        "lm_model_name_or_path": "/content/models/openbmb/MiniCPM-o-2_6-int4"
      }
    }
  },
  "config_list": []
}


In [43]:
!!DAILY_API_KEY=$DAILY_API_KEY \
  python -m src.cmd.bots.main -f /content/daily_minicpmo_vision_voice_bot.json


['{',
 '    "chat_bot_name": "DailyMiniCPMoVisionVoiceBot",',
 '    "config": {',
 '        "omni_llm": {',
 '            "args": {',
 '                "lm_attn_impl": "flash_attention_2",',
 '                "lm_device_map": "auto",',
 '                "lm_gen_temperature": 0.5,',
 '                "lm_model_name_or_path": "/content/models/openbmb/MiniCPM-o-2_6-int4",',
 '                "lm_torch_dtype": "bfloat16",',
 '                "use_gptq_ckpt": true,',
 '                "warnup_steps": 1',
 '            },',
 '            "tag": "minicpmo_vision_voice_processor"',
 '        },',
 '        "vad": {',
 '            "args": {',
 '                "stop_secs": 0.7',
 '            },',
 '            "tag": "silero_vad_analyzer"',
 '        }',
 '    },',
 '    "config_list": [],',
 '    "room_manager": {',
 '        "args": {',
 '            "privacy": "public"',
 '        },',
 '        "tag": "daily_room"',
 '    },',
 '    "room_name": "chat-room",',
 '    "room_url": "",',
 '  